# Uppgift 2 - Sportstatistik
- Välj 2-4 sporter och skapa lämpliga grafer/diagram för att visualisera exempelvis:
- medaljfördelning mellan länder i sporterna, åldersfördelning i sporterna
- Skapa fler plots för att visualisera olika aspekter kring sporterna.

### Våra frågeställningar:
- Kolla igenom vilka sporter som varit med?
- Analysera "Tug-Of-War"! (dragkamp) Kul att tex kolla hur länge det varit med i OS, vilka har varit bäst? Osv.
- Finns det andra gamla eller intressanta sporter att titta på?
- Alpinism?
- Aeronautics [https://www.topendsports.com/events/demonstration/aeronautics.htm](https://www.topendsports.com/events/demonstration/aeronautics.htm)
- Yngsta, äldsta medaljörerna?
#### "Tug of war" (Dragkamp) (Daniel):
- Vilka länder har deltagit och hur ser medaljfördelningen ut?
- Vilka år var detta en sport i OS?

#### Kulstötning / Slägga / Spjut / Diskus (Daniel):
- Hur ser längd- och vikt-utvecklingen ut genom historien?
- Analysera män och kvinnor separat
- Hade varit intressant att jämföra med resultaten i sporterna också....
- Distribution plots? Box-plots? Eller en animation frame, som går igenom åren?

In [1]:
import pandas as pd
import plotly.express as px
athlete_events = pd.read_csv("./Data/athlete_events.csv")

### 10 Bästa sporter för Frankrike

Här skapar jag en DataFrame med lämplig information för att plotta alla olika grafer som ska undersökas.

Informationen är:
- Totala medaljer vunnet av de top 10 sporterna.
- Av det totala, hur många av dem guld, silver och brons.
- Av varje guld silver och brons. Hur många var kvinnor och hur många var män.

In [76]:
only_france_df = athlete_events[athlete_events['Team'] == 'France']
only_france_df.head()
medal_counts = only_france_df.groupby('Sport')['Medal'].value_counts().unstack().fillna(0)
medal_counts[['Gold', 'Silver', 'Bronze']]
medal_counts['Total'] = medal_counts[['Gold', 'Silver', 'Bronze']].sum(axis=1)
medal_counts[['Gold', 'Silver', 'Bronze', 'Total']]
medal_counts['Participations'] = medal_counts.sum(axis=1)
medal_counts[['Gold', 'Silver', 'Bronze', 'Total', 'Participations']]
gold_female_counts = only_france_df[only_france_df['Medal'] == 'Gold'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Gold Female'] = gold_female_counts.fillna(0)
silver_female_counts = only_france_df[only_france_df['Medal'] == 'Silver'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Silver Female'] = silver_female_counts.fillna(0)

bronze_female_counts = only_france_df[only_france_df['Medal'] == 'Bronze'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Bronze Female'] = bronze_female_counts.fillna(0)

gold_male_counts = only_france_df[only_france_df['Medal'] == 'Gold'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Gold Male'] = gold_male_counts.fillna(0)

silver_male_counts = only_france_df[only_france_df['Medal'] == 'Silver'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Silver Male'] = silver_male_counts.fillna(0)

bronze_male_counts = only_france_df[only_france_df['Medal'] == 'Bronze'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Bronze Male'] = bronze_male_counts.fillna(0)

medal_counts['Total Female'] = medal_counts[['Gold Female', 'Silver Female', 'Bronze Female']].sum(axis=1)
medal_counts['Total Male'] = medal_counts[['Gold Male', 'Silver Male', 'Bronze Male']].sum(axis=1)
medal_counts.fillna(0, inplace=True)

medal_counts.head(10)

Medal,Bronze,Gold,Silver,Total,Participations,Gold Female,Silver Female,Bronze Female,Gold Male,Silver Male,Bronze Male,Total Female,Total Male
Sport,,,,,,,,,,,,,
Alpine Skiing,15.0,15.0,15.0,45.0,90.0,4.0,8.0,7.0,11.0,7.0,8.0,19.0,26.0
Archery,16.0,7.0,25.0,48.0,96.0,0.0,0.0,3.0,7.0,25.0,13.0,3.0,45.0
Art Competitions,5.0,6.0,4.0,15.0,30.0,0.0,1.0,0.0,6.0,3.0,5.0,1.0,14.0
Athletics,62.0,14.0,34.0,110.0,220.0,6.0,2.0,8.0,8.0,32.0,54.0,16.0,94.0
Basketball,0.0,0.0,38.0,38.0,76.0,0.0,12.0,0.0,0.0,26.0,0.0,12.0,26.0
Biathlon,24.0,8.0,8.0,40.0,80.0,4.0,5.0,10.0,4.0,3.0,14.0,19.0,21.0
Bobsleigh,4.0,0.0,0.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
Boxing,10.0,6.0,9.0,25.0,50.0,1.0,1.0,0.0,5.0,8.0,10.0,2.0,23.0
Canoeing,29.0,8.0,12.0,49.0,98.0,1.0,1.0,4.0,7.0,11.0,25.0,6.0,43.0


In [77]:
top_10_sports_total = medal_counts.nlargest(10, 'Total')
top_10_sports_total_plot = px.bar(top_10_sports_total, x=top_10_sports_total.index, y=['Bronze', 'Silver', 'Gold'], title='Top 10 sports with the highest total medals won by France', barmode='stack')

top_10_sports_total_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Total medals won",
)
top_10_sports_total_plot.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
top_10_sports_total_plot.update_traces(marker_color='silver', selector=dict(name='Silver'))
top_10_sports_total_plot.update_traces(marker_color='gold', selector=dict(name='Gold'))
top_10_sports_total_plot.show()

In [81]:
top_10_sports_gold = medal_counts.nlargest(10, 'Gold')
top_10_sports_gold_plot = px.bar(top_10_sports_gold, x=top_10_sports_gold.index, y=['Gold Female', 'Gold Male'], title='Top 10 sports with the highest number of gold medals won by France', barmode='stack')
top_10_sports_gold_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of gold medals won"
)
top_10_sports_gold_plot.show()

In [82]:
top_10_sports_silver = medal_counts.nlargest(10, 'Silver')
top_10_sports_silver_plot = px.bar(top_10_sports_silver, x=top_10_sports_silver.index, y=['Silver Female', 'Silver Male'], title='Top 10 sports with the highest number of silver medals won by France', barmode='stack')
top_10_sports_silver_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of silver medals won"
)
top_10_sports_silver_plot.show()

In [83]:
top_10_sports_bronze = medal_counts.nlargest(10, 'Bronze')
top_10_sports_bronze_plot = px.bar(top_10_sports_bronze, x=top_10_sports_bronze.index, y=['Bronze Female', 'Bronze Male'], title='Top 10 sports with the highest number of bronze medals won by France', barmode='stack')
top_10_sports_bronze_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of bronze medals won"
)
top_10_sports_bronze_plot.show()

### Histogram över olika åldrar över tid

In [95]:
# Import the necessary libraries
import pandas as pd

# Filter the dataframe to include only rows where the Team is 'France'
only_france_df = athlete_events[athlete_events['Team'] == 'France']

# Create a new dataframe with grouped age ranges
only_france_df['Age_Range'] = (only_france_df['Age'] // 10) * 10
only_france_df['Age_Range'] = only_france_df['Age_Range'].astype(str) + '-' + (only_france_df['Age_Range'] + 9).astype(str)

age_range_counts = only_france_df.groupby(['Year', 'Age_Range']).size().reset_index(name='Count')

# Reshape the dataframe to make it more readable
age_range_counts_pivot = age_range_counts.pivot(index='Year', columns='Age_Range', values='Count')
age_range_counts_pivot.fillna(0, inplace=True)

age_range_counts_pivot


/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_53225/148471355.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_53225/148471355.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Age_Range,10.0-19.0,20.0-29.0,30.0-39.0,40.0-49.0,50.0-59.0,60.0-69.0,70.0-79.0,80.0-89.0,nan-nan
Year,,,,,,,,,
1896,11.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
1900,37.0,109.0,77.0,47.0,5.0,0.0,0.0,0.0,404.0
1904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1906,5.0,39.0,15.0,28.0,20.0,0.0,0.0,0.0,42.0
1908,19.0,78.0,34.0,17.0,9.0,0.0,0.0,0.0,136.0
1912,8.0,104.0,55.0,25.0,14.0,2.0,0.0,0.0,2.0
1920,33.0,131.0,84.0,55.0,19.0,4.0,0.0,0.0,111.0
1924,38.0,249.0,140.0,76.0,11.0,6.0,7.0,1.0,148.0
1928,20.0,153.0,64.0,39.0,1.0,15.0,2.0,0.0,154.0
